# Imports

In [2]:
%load_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/rodrigoeji/sandbox/keys/galena-web-adea43fba8c1.json"

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import *

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# np.set_printoptions(threshold=1000)

In [73]:
%%bigquery afs
select applicant_id,
# logic_test_score,
calendly_event_starts_at,
cast(current_date as date) as communication_date
from `galena-web.inclusion.applicant_funnel_snapshot`
where (
        (
            (date_diff(cast(calendly_event_starts_at as date), cast(current_date as date), day) = 3) 
            and (extract(dayofweek from cast(calendly_event_starts_at as date)) = 2)
        )
            or date_diff(cast(calendly_event_starts_at as date), cast(current_date as date), day) = 1
      ) and calendly_event_starts_at is not null

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 14.22rows/s]


# Sampling

In [74]:
test = afs.sample(frac=0.5, replace=False, random_state=0)
test['group'] = 'test'
control = afs.merge(test[['applicant_id', 'group']], on = 'applicant_id', how = 'left')
control = control[control['group'] != 'test']
control['group'] = 'control'

In [97]:
filename_t = 'test_' + date.today().strftime("%d-%m-%Y")+'.csv'
filename_c = 'control_' + date.today().strftime("%d-%m-%Y")+'.csv'

In [101]:
test.to_csv(filename_t)
control.to_csv(filename_c)

# xx